In [6]:
# Importer les bibliothèques nécessaires
import time
import random
from pymongo import MongoClient
from cassandra.cluster import Cluster

In [7]:
# Connexion à MongoDB
mongo_client = MongoClient('localhost', 27017)
mongo_db = mongo_client.test_db
mongo_collection = mongo_db.compare_collection

In [11]:
# Connexion à Cassandra
cassandra_cluster = Cluster(['127.0.0.1'])
cassandra_session = cassandra_cluster.connect()

# Créer un Keyspace et une Table
cassandra_session.execute("""
CREATE KEYSPACE IF NOT EXISTS compare_keyspace
WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 3}
""")
cassandra_session.execute("USE compare_keyspace")
cassandra_session.execute("""
CREATE TABLE IF NOT EXISTS test_table (
    id int PRIMARY KEY,
    nom text,
    code int
)
""")

In [ ]:
cassandra_session.set_keyspace('compare_keyspace')

In [20]:
# Charger des données etudiant dans MongoDB
def load_data_mongodb():
    for i in range(1000):
        mongo_collection.insert_one({
            'id': i,
            'nom': f'User {i}',
            'code': random.randint(1, 1000)
        })

In [21]:
# Charger des données etudiant dans Cassandra
def load_data_cassandra():
    for i in range(1000):
        cassandra_session.execute("""
        INSERT INTO test_table (id, nom, code)
        VALUES (%s, %s, %s)
        """, (i, f'User {i}', random.randint(1, 1000)))

In [22]:
# Mesurer les performances d'écriture
def write_performance():
    # MongoDB
    start_time = time.time()
    for i in range(1000, 2000):
        mongo_collection.insert_one({
            'id': i,
            'nom': f'User {i}',
            'code': random.randint(1, 1000)
        })
    mongo_write_time = time.time() - start_time
    # Cassandra
    start_time = time.time()
    for i in range(1000, 2000):
        cassandra_session.execute("""
        INSERT INTO test_table (id, nom, code)
        VALUES (%s, %s, %s)
        """, (i, f'User {i}', random.randint(1, 1000)))
    cassandra_write_time = time.time() - start_time

    print(f"Temps d'écriture dans MongoDB: {mongo_write_time} secondes")
    print(f"Temps d'écriture dans Cassandra: {cassandra_write_time} secondes")

In [23]:
# Mesurer les performances de lecture
def read_performance():
    # MongoDB
    start_time = time.time()
    for i in range(2000):
        mongo_collection.find_one({'id': i})
    mongo_read_time = time.time() - start_time

    # Cassandra
    start_time = time.time()
    for i in range(2000):
        cassandra_session.execute("SELECT * FROM test_table WHERE id=%s", (i,))
    cassandra_read_time = time.time() - start_time

    print(f"Temps de lecture dans MongoDB: {mongo_read_time} secondes")
    print(f"Temps de lecture dans Cassandra: {cassandra_read_time} secondes")


In [24]:
# Charger les données dans les deux bases de données
load_data_mongodb()
load_data_cassandra()

# Mesurer les performances
write_performance()
read_performance()

Temps d'écriture dans MongoDB: 0.09858202934265137 secondes
Temps d'écriture dans Cassandra: 0.261552095413208 secondes
Temps de lecture dans MongoDB: 0.49334716796875 secondes
Temps de lecture dans Cassandra: 0.67488694190979 secondes
